In [27]:
using Plots
using LinearAlgebra
using SpecialFunctions

In [132]:
#exponents = [3.425250914, 0.6239137298, 0.1688554040]


# Helpful structs
struct Gaussian
    exponent::Number
    center::AbstractArray{<:Number, 1}
end

struct Atom
    charge::Number
    position::AbstractArray{<:Number, 1}
end
#############

# Integral functions
function overlap_integral(A::T, B::T) where T<:Gaussian
    α = A.exponent
    β = B.exponent
    R_A = A.center
    R_B = B.center
    
    return (4*α*β / (α + β) ^ 2) ^ (3/4) * exp(-α*β / (α + β) * norm(R_A - R_B)^2)
end


function kinetic_integral(A::T, B::T) where T<:Gaussian
    α = A.exponent
    β = B.exponent
    R_A = A.center
    R_B = B.center
    
    a = α * β / (α + β)
    
    return overlap_integral(A, B) * a * (3 - 2 * a * norm(R_A - R_B)^2)
end

function kinetic_integral(arr::Tuple{T, T}) where T <: Gaussian
    return kinetic_integral(arr[1], arr[2])
end


function core_potential_integral(A::T, B::T, C::Atom) where T<:Gaussian
    α = A.exponent
    β = B.exponent
    R_A = A.center
    R_B = B.center
    R_C = C.position
    Z = C.charge
        
    b = sqrt(α + β)
    
    R_P = (α * R_A + β * R_B) / (α + β)
    
    if R_P == R_C
        return -overlap_integral(A, B) * Z * 2 * b / sqrt(π)
    else
        return -overlap_integral(A, B) * Z / norm(R_P - R_C) * erf(b * norm(R_P - R_C))
    end
end

function core_potential_integral(A::T, B::T, atoms::AbstractArray{Atom,1}) where T<:Gaussian
    return sum(core_potential_integral(A, B, a) for a in atoms)
end

function core_potential_integral(arr::Tuple{T, T}, atoms::AbstractArray{Atom,1}) where T<:Gaussian
    return core_potential_integral(arr[1], arr[2], atoms)
end


function electron_electron_potential(A::T, B::T, C::T, D::T) where T<:Gaussian
    α = A.exponent
    β = B.exponent
    γ = C.exponent
    δ = D.exponent
    
    R_A = A.center
    R_B = B.center
    R_C = C.center
    R_D = D.center
    
    R_P = (α * R_A + β * R_B) / (α + β)
    R_Q = (γ * R_C + δ * R_D) / (γ + δ)
    
    c = sqrt((α + β) * (γ + δ) / (α + β + γ + δ))
    
    if R_P == R_Q 
        return overlap_integral(A, B) * overlap_integral(C, D) * 2 * c / sqrt(π)
    else
        return overlap_integral(A, B) * overlap_integral(C, D) / norm(R_P - R_Q) * erf(c * norm(R_P - R_Q))
    end
end

function electron_electron_potential(arr::NTuple{4, Gaussian})
    return electron_electron_potential(arr[1], arr[2], arr[3], arr[4])
end


function exchange_integral(A::T, B::T, C::T, D::T) where T<:Gaussian
    return electron_electron_potential(A, C, B, D)
end

function exchange_integral(arr::NTuple{4, Gaussian})  
    return exchange_integral(arr[1], arr[2], arr[3], arr[4])
end

#################

exchange_integral (generic function with 3 methods)

In [107]:
# Reading molecular info from file
positions = []
exponents = []
open("mols.txt", "r") do mol_file
    while !eof(mol_file)
        line = readline(mol_file)
        if line == "COORDINATES:"
            while line != "==="
                line = readline(mol_file)
                if line == "==="
                    break
                end
                push!(positions, map(x->parse(Float64, x), split(line))) 
            end
        end
        if line == "EXPONENTS:"
            while line != "==="
                line = readline(mol_file)
                if line == "==="
                    break
                end
                push!(exponents, tryparse(Float64, line)) 
            end
        end
    end
end

atoms = Array{Atom}(undef, 3)
gaussians = Array{Gaussian}(undef, 6)

n = 1
for e in exponents
    for p in positions
        if n <= 3
            atoms[n] = Atom(1, p)
        end
        gaussians[n] = Gaussian(e, p)
        n += 1
    end
end

In [133]:
# Assemble matrices

one_body = Iterators.product(gaussians, gaussians)
T = map(x->kinetic_integral(x), one_body)
V = map(x->core_potential_integral(x, atoms), one_body)

H = T + V

two_body = Iterators.product(gaussians, gaussians, gaussians, gaussians)
M0 = map(x->electron_electron_potential(x), two_body)
M1 = map(x->exchange_integral(x), two_body)
M = M0 - 1/2 * M1

6×6×6×6 Array{Float64,4}:
[:, :, 1, 1] =
 0.272414   0.207666  0.158494    0.204666    0.117328    0.0686972
 0.207666   0.299894  0.23025     0.12111     0.223451    0.133351
 0.158494   0.23025   0.306772    0.0688829   0.141459    0.220704
 0.204666   0.12111   0.0688829   0.527787    0.0906399  -0.00170208
 0.117328   0.223451  0.141459    0.0906399   0.477303    0.103127
 0.0686972  0.133351  0.220704   -0.00170208  0.103127    0.425029

[:, :, 2, 1] =
 0.207666   0.170895   0.130276    0.152929   0.106028   0.061011
 0.106396   0.207666   0.158494    0.0473168  0.168839   0.098077
 0.0802896  0.158494   0.23025     0.0176097  0.102409   0.177759
 0.168839   0.106028   0.0626524   0.406122   0.0859635  0.00427382
 0.0473168  0.152929   0.090233    0.0320752  0.406122   0.0744841
 0.0248412  0.0883408  0.169418   -0.0268586  0.0744841  0.36614

[:, :, 3, 1] =
  0.158494    0.130276   0.107209    0.114585   0.0793328  0.0549649
  0.0802896   0.158494   0.130276    0.03429    0.12676

In [109]:
+(1,2,3)

6

In [80]:
println("Core potential integrals")
for g1 in gaussians
    for g2 in gaussians
        V = sum(core_potential_integral(g1, g2, a) for a in atoms)
        print(V, " ")
    end
    println()
end

println("\nKinetic integrals")
for g1 in gaussians
    for g2 in gaussians
        T = kinetic_integral(g1, g2)
        print(T, " ")
    end
end

println("\nElectron-electron integrals")
for (i, g1) in enumerate(gaussians)
    for (j, g2) in enumerate(gaussians)
        for (k, g3) in enumerate(gaussians)
            for (l, g4) in enumerate(gaussians)
                M0 = electron_electron_potential(g1, g2, g3, g4)
                M1 = exchange_integral(g1, g2, g3, g4)
                println("($(i)$(j)|$(k)$(l)) ", M0 - 1/2 * M1, " ")
            end
        end
    end
end

Core potential integrals
-1.8292418533364327 -1.5693989053463677 -1.2530370193175042 -1.5757891780403996 -1.1687303705303052 -0.7470001465781908 
-1.5693989053463677 -1.9414177711685445 -1.5693989053463677 -1.1065702118840823 -1.6959138130822509 -1.1065702118840826 
-1.2530370193175042 -1.5693989053463677 -1.8292418533364327 -0.7470001465781908 -1.1687303705303052 -1.5757891780403996 
-1.5757891780403996 -1.1065702118840823 -0.7470001465781908 -3.032522075213852 -0.8534150772364473 -0.2136887290079682 
-1.1687303705303052 -1.6959138130822509 -1.1687303705303052 -0.8534150772364473 -3.23877701538122 -0.8534150772364473 
-0.7470001465781908 -1.1065702118840826 -1.5757891780403996 -0.2136887290079682 -0.8534150772364473 -3.032522075213852 

Kinetic integrals
0.349704 0.23393201282889098 0.15118763172811203 0.35993729404619645 0.17835104024435547 0.07701784717108676 0.23393201282889098 0.349704 0.23393201282889098 0.17835104024435547 0.35993729404619645 0.17835104024435547 0.15118763172811

COORDINATES:
-1 0 0
0 1 0
1 0 0

EXPONENTS:
0.233136
1.309757
Any[[-1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [1.0, 0.0, 0.0]]
Any[0.233136, 1.309757]


In [70]:
map(x->parse(Float64, x), split("1 0 0"))

3-element Array{Float64,1}:
 1.0
 0.0
 0.0